# Setup of data and repos for project

In [5]:
# Installation
#!pip install -U scikit-learn
# !pip install --upgrade gensim

In [53]:
# Importing data
import json

# Smart lights
File = open("openvoc-keyword-spotting-research-datasets/smart-lights/metadata.json")
metaData_in = json.load(File)
N =  len(metaData_in)
keys = list(metaData_in.keys())

i = 150
print(metaData_in.get(keys[i]).get('transcript'))
print(metaData_in.get(keys[i]).get('keywords'))

# Washing machine
# ...

can you turn on lights for the kitchen
['turn on', 'kitchen']
